In [1]:
## IMPORTS 

import os
import pandas as pd

In [33]:
## BASE HISTORICA 

# Definir o diretório contendo os arquivos CSV
diretorio = "challenge-webmedia-e-globo-2023/files/treino"

# Listar todos os arquivos CSV na pasta
arquivos_csv = [f for f in os.listdir(diretorio) if f.endswith(".csv")]

# Criar uma lista para armazenar os DataFrames
dataframes = []

# Loop através dos arquivos CSV e ler cada um
for arquivo in arquivos_csv:
    caminho_completo = os.path.join(diretorio, arquivo)
    df = pd.read_csv(caminho_completo)
    dataframes.append(df)

# Concatenar todos os DataFrames em um único DataFrame
treino = pd.concat(dataframes, ignore_index=True)

# Exibir as primeiras linhas do DataFrame
treino.head()


,userId,userType,historySize,history,timestampHistory,numberOfClicksHistory,timeOnPageHistory,scrollPercentageHistory,pageVisitsCountHistory,timestampHistory_new
0,fbb963d61eb8149e7f43b1bd905457ba5e106a830ddc27...,Non-Logged,2,"80aa7bb2-adce-4a55-9711-912c407927a1, d9e5f15d...","1657908085200, 1659634203762","0, 0","71998, 115232","81.58, 73.36","1, 1","1657908085200, 1659634203762"
1,17f1083e6079b0f28f7820a6803583d1c1b405c0718b11...,Non-Logged,2,"19ba89fc-1e06-4c5d-9c57-4a3088dc0511, e273dba4...","1657111508570, 1657481309920","68, 12","131495, 43733","51.74, 35.49","1, 1","1657111508570, 1657481309920"
2,528a8d7a2af73101da8d6709c1ec875b449a5a58749a99...,Non-Logged,2,"59a61a8a-cc52-453f-b1cd-2bd019e9d574, a0562805...","1657823890328, 1660141444328","55, 9","159042, 10336","62.19, 48.28","1, 1","1657823890328, 1660141444328"
3,2dd18b58a634a4e77181a202cf152df6169dfb3e4230ef...,Non-Logged,2,"233f8238-2ce0-470f-a9d5-0e0ac530382a, 037155f4...","1656963373076, 1657091888917","0, 0","193579, 20519","31.03, 31.9","1, 1","1656963373076, 1657091888917"
4,97e1439d485b0630e12818d3df84ff67d08475ef6ebeb0...,Logged,2,"385044ad-3876-4188-83fa-f560435c1a9c, 2f754502...","1657618607633, 1659536839832","57, 38","220000, 130000","52.65, 53.37","1, 1","1657618607633, 1659536839832"


In [34]:
# Criar uma lista para armazenar os dados processados para todos os usuários
processed_data = []

# Iterar sobre cada usuário no DataFrame original
for _, row in df.iterrows():
    history = row["history"].split(", ") if pd.notna(row["history"]) else []
    clicks = list(map(int, row["numberOfClicksHistory"].split(", "))) if pd.notna(row["numberOfClicksHistory"]) else []
    time_on_page = list(map(int, row["timeOnPageHistory"].split(", "))) if pd.notna(row["timeOnPageHistory"]) else []
    scroll_percentage = list(map(float, row["scrollPercentageHistory"].split(", "))) if pd.notna(row["scrollPercentageHistory"]) else []

    # Criar um DataFrame temporário com os dados históricos do usuário
    history_df = pd.DataFrame({
        "history": history[:len(clicks)],  # Garantindo que os tamanhos batam
        "clicks": clicks,
        "page_time": time_on_page,
        "scroll_percentage": scroll_percentage
    })

    # Criar a métrica de engajamento para cada página
    history_df["engajamento"] = (history_df["clicks"] * 0.4) + (history_df["page_time"] * 0.4) + (history_df["scroll_percentage"] * 0.2)

    # Ordenar pelo maior engajamento e selecionar os top 5
    top_5_history = history_df.sort_values(by="engajamento", ascending=False).head(5)

    # Criar a estrutura de features
    features = {
        "top_5_history": top_5_history.to_dict(orient="records")
    }

    # Adicionar ao dataset processado
    processed_data.append({
        "userId": row["userId"],
        "features": features
    })

# Criar um novo DataFrame com todos os usuários processados
treino_feature = pd.DataFrame(processed_data)


In [35]:
treino_feature.head(10)

,userId,features
0,e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d...,{'top_5_history': [{'history': '458bf0ec-efb4-...
1,3a3b7f25a30a5a17a530685545e3a0be38cee0c6904c42...,{'top_5_history': [{'history': 'bacab81b-4dd0-...
2,ea6728ebc30782516a7593a1143c47dc59428e1649c048...,{'top_5_history': [{'history': 'bd1bc41b-79a2-...
3,f909c50558c01ab790636e1b1918e6a5965bdcc271a860...,{'top_5_history': [{'history': 'ec195f2d-9d6e-...
4,9725352f759a73aa977cb5eff2596a5f683fcd0c753200...,{'top_5_history': [{'history': 'efef6019-5180-...
5,be4093de9e85714d07ef90e1bea19e60890b4e4b43103f...,{'top_5_history': [{'history': '6959aea6-c6e1-...
6,b2360a3e91d21168d54c4a0c094f8aa4f605135266c836...,{'top_5_history': [{'history': 'a22a185a-474f-...
7,42829ee6ae5174c90f8272cbac965d348b5b174e65e00c...,{'top_5_history': [{'history': '61e07f64-cddf-...
8,9ac2c69a8098084173932abbb6a4617f537f0633e070c3...,{'top_5_history': [{'history': 'b93c6798-2b44-...
9,500fab2f0eb549a0144d82ac17dd0ee583e548e71481bf...,{'top_5_history': [{'history': 'c309c7aa-b438-...


In [36]:
# Exemplo: Selecionar um userId específico para visualizar a coluna "features"
user_id_especifico = treino_feature.iloc[3]["userId"]  # Pegando o primeiro usuário como exemplo
conteudo_feature = treino_feature[treino_feature["userId"] == user_id_especifico]["features"].values[0]

conteudo_feature

{'top_5_history': [{'history': 'ec195f2d-9d6e-4c33-a6f1-671036ffbf6a',
   'clicks': 30,
   'page_time': 105697,
   'scroll_percentage': 44.87,
   'engajamento': 42299.774000000005},
  {'history': 'fbe1e6a7-401b-439f-8bc3-908bddad0de1',
   'clicks': 0,
   'page_time': 51977,
   'scroll_percentage': 12.5,
   'engajamento': 20793.300000000003},
  {'history': 'c923f085-d480-4c64-9d9f-f53d368da37f',
   'clicks': 0,
   'page_time': 48880,
   'scroll_percentage': 12.5,
   'engajamento': 19554.5},
  {'history': '92165aa2-a803-4d0f-8d2b-44977f224046',
   'clicks': 0,
   'page_time': 43153,
   'scroll_percentage': 12.5,
   'engajamento': 17263.7},
  {'history': 'f0a78e58-ec7e-494c-9462-fbd6446a9a89',
   'clicks': 0,
   'page_time': 41626,
   'scroll_percentage': 12.5,
   'engajamento': 16652.9}]}

In [37]:
treino_feature.to_excel("treino_feature.xlsx", index=False)


In [38]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from ast import literal_eval

# Carregar os dados
df = pd.read_excel("treino_feature.xlsx")

# Converter a coluna 'features' de string para dicionário
df["features"] = df["features"].apply(literal_eval)

# Extrair métricas relevantes para o modelo
feature_data = []
history_mapping = {}
for _, row in df.iterrows():
    user_id = row["userId"]
    top_5_history = row["features"].get("top_5_history", [])
    
    clicks = [h.get("clicks", 0) for h in top_5_history]
    page_time = [h.get("page_time", 0) for h in top_5_history]
    scroll_percentual = [h.get("scroll_percentual", h.get("scrollPercentage", 0)) for h in top_5_history]
    history = [h.get("history", "") for h in top_5_history]
    
    feature_data.append([
        user_id,
        np.mean(clicks), np.max(clicks), np.sum(clicks),
        np.mean(page_time), np.max(page_time), np.sum(page_time),
        np.mean(scroll_percentual), np.max(scroll_percentual)
    ])
    
    history_mapping[user_id] = history  # Guardar o histórico de cada usuário

# Criar DataFrame de features
columns = ["userId", "mean_clicks", "max_clicks", "sum_clicks",
           "mean_page_time", "max_page_time", "sum_page_time",
           "mean_scroll", "max_scroll"]
features_df = pd.DataFrame(feature_data, columns=columns)

# Separar userId para referência futura
user_ids = features_df["userId"]
X = features_df.drop(columns=["userId"])

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, user_ids, test_size=0.2, random_state=42)

# Treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=6, metric='euclidean')
knn.fit(X_train, y_train)

# Função para recomendar páginas baseado nos 6 perfis mais similares
def recomendar_paginas(user_id):
    if user_id not in history_mapping:
        return "Usuário não encontrado."
    
    user_index = user_ids[user_ids == user_id].index[0]
    user_features = X_scaled[user_index].reshape(1, -1)
    neighbors = knn.kneighbors(user_features, n_neighbors=6, return_distance=False)[0]
    
    recomendacoes = set()
    for neighbor in neighbors:
        neighbor_id = user_ids.iloc[neighbor]
        recomendacoes.update(history_mapping.get(neighbor_id, []))
    
    return list(recomendacoes)[:6]  # Retorna as top 6 páginas recomendadas dos usuários mais similares

# Teste com um userId específico
sample_user_id = user_ids.iloc[0]
recomendacoes = recomendar_paginas(sample_user_id)
print(f"Recomendações para o usuário {sample_user_id}: {recomendacoes}")


Recomendações para o usuário e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d5092adc513f516b377: ['2c21cb58-6ac6-4618-9a53-a27871a7c276', '0a780d88-b16c-40a2-abec-52b5653a1eb6', '16bd7777-abb4-4a50-a51e-df8e5f7e4a5d', 'aa6ca24c-053f-4cb0-9bdc-05d8b6def58a', '6767b1cc-fffc-4eb1-8d66-e0f616a8c9e8', '01bc9330-2841-46e0-a13b-a1fad9a66148']


In [39]:
user_id = "760e33b6b680b88dea00dea6730ee654e3038075685f9a811f0a7ce910145d52"
recomendacoes = recomendar_paginas(user_id)
print(f"Recomendações para o usuário {user_id}: {recomendacoes}")



Recomendações para o usuário 760e33b6b680b88dea00dea6730ee654e3038075685f9a811f0a7ce910145d52: ['756f1fd7-703f-4ba3-ac49-a7c9bff1b1ec', 'c11a28a1-e5bf-4690-a648-fe8e993607d4', '5468ea94-7385-4e7c-b5b6-4cd13cf58e80', '8e0884e2-50ba-44c0-87b9-d64d913288af', '03e43166-56a9-48c2-a005-c0b578b78395', '4c63d7cd-4902-4ffb-9b94-578b1b2151f0']


In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from ast import literal_eval

# Carregar os dados
df = pd.read_excel("treino_feature.xlsx")

# Converter a coluna 'features' de string para dicionário
df["features"] = df["features"].apply(literal_eval)

# Extrair métricas relevantes para o modelo
feature_data = []
history_mapping = {}
for _, row in df.iterrows():
    user_id = row["userId"]
    top_5_history = row["features"].get("top_5_history", [])
    
    clicks = [h.get("clicks", 0) for h in top_5_history]
    page_time = [h.get("page_time", 0) for h in top_5_history]
    scroll_percentual = [h.get("scroll_percentual", h.get("scrollPercentage", 0)) for h in top_5_history]
    history = [h.get("history", "") for h in top_5_history]
    
    feature_data.append([
        user_id,
        np.mean(clicks), np.max(clicks), np.sum(clicks),
        np.mean(page_time), np.max(page_time), np.sum(page_time),
        np.mean(scroll_percentual), np.max(scroll_percentual),
        len(history)  # Quantidade de interações como proxy para cold-start
    ])
    
    history_mapping[user_id] = history  # Guardar o histórico de cada usuário

# Criar DataFrame de features
columns = ["userId", "mean_clicks", "max_clicks", "sum_clicks",
           "mean_page_time", "max_page_time", "sum_page_time",
           "mean_scroll", "max_scroll", "history_size"]
features_df = pd.DataFrame(feature_data, columns=columns)

# Separar userId para referência futura
user_ids = features_df["userId"]
X = features_df.drop(columns=["userId"])

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, user_ids, test_size=0.2, random_state=42)

# Treinar o modelo KNN
knn = KNeighborsClassifier(n_neighbors=6, metric='euclidean')
knn.fit(X_train, y_train)

# Função para recomendar páginas baseado nos 6 perfis mais similares
def recomendar_paginas(user_id):
    if user_id not in history_mapping or len(history_mapping[user_id]) == 0:
        # Cold-start para usuário novo: recomenda páginas mais populares
        all_histories = sum(history_mapping.values(), [])
        popular_pages = pd.Series(all_histories).value_counts().index[:6].tolist()
        return popular_pages
    
    user_index = user_ids[user_ids == user_id].index[0]
    user_features = X_scaled[user_index].reshape(1, -1)
    neighbors = knn.kneighbors(user_features, n_neighbors=6, return_distance=False)[0]
    
    recomendacoes = set()
    for neighbor in neighbors:
        neighbor_id = user_ids.iloc[neighbor]
        recomendacoes.update(history_mapping.get(neighbor_id, []))
    
    return list(recomendacoes)[:6]  # Retorna as top 6 páginas recomendadas dos usuários mais similares

# Teste com um userId específico
sample_user_id = user_ids.iloc[0]
recomendacoes = recomendar_paginas(sample_user_id)
print(f"Recomendações para o usuário {sample_user_id}: {recomendacoes}")


Recomendações para o usuário e5f68d5e7cdbe56d6984589b4baa6ebfc5e8a8a918e57d5092adc513f516b377: ['0a780d88-b16c-40a2-abec-52b5653a1eb6', 'd54c05ca-5fe9-467b-810a-d2e307256173', '2c21cb58-6ac6-4618-9a53-a27871a7c276', '16bd7777-abb4-4a50-a51e-df8e5f7e4a5d', 'aa180263-187b-4ef7-96b2-3101077b4f8d', '01bc9330-2841-46e0-a13b-a1fad9a66148']


In [6]:
new_user_id = "760e33b6b680b88dea00dea6730ee654e3038075685f9a811f0a7ce910000000"
recomendacoes = recomendar_paginas(new_user_id)
print(f"Recomendações para novo usuário: {recomendacoes}")

Recomendações para novo usuário: ['756f1fd7-703f-4ba3-ac49-a7c9bff1b1ec', '85beb942-01d5-44ce-a5aa-296139c73667', 'a9c454c8-9493-4622-a499-43d20c525f81', '5468ea94-7385-4e7c-b5b6-4cd13cf58e80', '8d3377c0-7e1e-46a7-8217-3009257ec202', 'd51075be-bc8e-4389-b7fc-4ac2a07bd669']
